# Implementation of the Kuribayashi BERT minus model

## libraries

In [1]:
!pip install transformers --upgrade
!pip install ipywidgets
!pip install IProgress
!pip install datasets
!pip install torch-lr-finder

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 5.8 MB 20.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.6 MB 59.0 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.12.5
    Uninstalling transformers-4.12.5:
      Successfully uninstalled transformers-4.12.5
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertModel, BertForSequenceClassification
from transformers import BatchEncoding, default_data_collator, DataCollatorWithPadding

import torch
import torch.nn as nn

import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import datasets

from torch.utils.data import DataLoader

from tqdm import tqdm
from operator import itemgetter

In [3]:
print(transformers.__version__)

4.25.1


## tokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
# tokenizer.model_max_length = 256

## data

In [6]:
DATA_FILE = '/notebooks/KURI-BERT/notebooks/full_formula_w_fts/CLS_work/pe_dataset_for_cls_implementation.pt'

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load data

In [8]:
dataset = torch.load(DATA_FILE)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'structural_fts_as_text_combined', 'paragraph_labels_numeric', 'full_paragraph_w_cls', 'component_cls_indices', 'feature_cls_indices', 'sanity_1'],
        num_rows: 1088
    })
    test: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'structural_fts_as_text_combined', 'paragraph_labels_numeric', 'full_paragraph_w_cls', 'component_cls_indices', 'feature_cls_indices', 'sanity_1'],
        num_rows: 358
    })
    validation: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'structural_fts_as_text_combined', 'paragraph_labels_numeric', 'full_paragraph_w_cls', 'component_cls_indices', 'feature_cls_indices', 'sanity_1'],
        num_rows:

In [10]:
dataset['train']['full_paragraph_w_cls'][498]

'[CLS] Computers have revolutionized our lifestyle during last century. However, some people especially young ones use computers extremely and this excessive usage lead to increase stress among them, but computers have brought many advantages to our lives. From my point of view, computers have an important and inevitable role in economic, business, and other part of societies. [CLS] 1, Yes, No, Yes, No [CLS] In addition, they make the way of living easier by doing our routine duties. [CLS] 1, No, Yes, Yes, No'

In [11]:
dataset['train']['component_cls_indices'][498]

[0, 76]

In [12]:
dataset['train']['feature_cls_indices'][498]

[66, 93]

In [13]:
for i,x in enumerate(tokenizer.tokenize(dataset['train']['full_paragraph_w_cls'][498])):
    print(i, x)

0 [CLS]
1 computers
2 have
3 revolution
4 ##ized
5 our
6 lifestyle
7 during
8 last
9 century
10 .
11 however
12 ,
13 some
14 people
15 especially
16 young
17 ones
18 use
19 computers
20 extremely
21 and
22 this
23 excessive
24 usage
25 lead
26 to
27 increase
28 stress
29 among
30 them
31 ,
32 but
33 computers
34 have
35 brought
36 many
37 advantages
38 to
39 our
40 lives
41 .
42 from
43 my
44 point
45 of
46 view
47 ,
48 computers
49 have
50 an
51 important
52 and
53 inevitable
54 role
55 in
56 economic
57 ,
58 business
59 ,
60 and
61 other
62 part
63 of
64 societies
65 .
66 [CLS]
67 1
68 ,
69 yes
70 ,
71 no
72 ,
73 yes
74 ,
75 no
76 [CLS]
77 in
78 addition
79 ,
80 they
81 make
82 the
83 way
84 of
85 living
86 easier
87 by
88 doing
89 our
90 routine
91 duties
92 .
93 [CLS]
94 1
95 ,
96 no
97 ,
98 yes
99 ,
100 yes
101 ,
102 no


### preprocessing

In [14]:
MAX_LENGTH = 0
MAX_CLS_IDX = 0

for split in ['train', 'test', 'validation']:
    
    for col_name in ['component_cls_indices', 'feature_cls_indices']:
        
        for x in dataset[split][col_name]:
            
            if max(x) > MAX_CLS_IDX:
                
                MAX_CLS_IDX = max(x)
                MAX_CLS_IDX = min(MAX_CLS_IDX, tokenizer.model_max_length - 2)
            
            if len(x) > MAX_LENGTH:
                
                MAX_LENGTH = len(x)

In [15]:
MAX_LENGTH

12

In [16]:
MAX_CLS_IDX

304

In [17]:
def get_padding(batch, padding_target):    
    
    if padding_target == 'adu_cls_indices':
        
        col_name = 'component_cls_indices'
        padding_val = [-1]
        max_length = MAX_LENGTH
        
        
    elif padding_target == 'features_cls_indices':
        
        col_name = 'feature_cls_indices'
        padding_val = [-1]
        max_length = MAX_LENGTH
        
    elif padding_target == 'label':
    
        col_name = 'paragraph_labels_numeric'
        padding_val = [-100]      
        max_length = MAX_LENGTH

    padded_indices_ll = []

    for i, indices_l in enumerate(batch[col_name]):

        padded_indices_l = indices_l + (max_length - len(indices_l)) * padding_val
        padded_indices_ll.append(padded_indices_l)

    return padded_indices_ll        

In [18]:
def get_cls_indices(adu_cls_indices_ll, features_cls_indices_ll):
    
    
    indices_ll = []
    
    for adu_cls_indices_l, features_cls_indices_l in zip(adu_cls_indices_ll, features_cls_indices_ll):


        indices_l = [[i,j] for i, j in zip(adu_cls_indices_l, features_cls_indices_l)]        
        indices_ll.append(indices_l)

    return indices_ll

### tokenize 

In [19]:
def tokenize(batch):
    
    tokenized_text = tokenizer(batch['full_paragraph_w_cls'], truncation=True, padding=True, max_length=512)
    tokenized_text['label'] = get_padding(batch, 'label')
    tokenized_text['adu_cls_indices'] = get_padding(batch, 'adu_cls_indices')
    tokenized_text['features_cls_indices'] = get_padding(batch, 'features_cls_indices')
    #tokenized_text['fts_spans'] = get_padding(batch, 'fts_spans')
    tokenized_text['cls_indices'] = get_cls_indices(tokenized_text['adu_cls_indices'], tokenized_text['features_cls_indices'])      
    
    return tokenized_text

In [20]:
dataset = dataset.map(tokenize, batched=True, batch_size=len(dataset['train']))

Parameter 'function'=<function tokenize at 0x7fc70ede8700> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['adu_cls_indices', 'attention_mask', 'cls_indices', 'component_cls_indices', 'essay_nr', 'feature_cls_indices', 'features_cls_indices', 'full_paragraph_w_cls', 'input_ids', 'label', 'paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_labels_numeric', 'paragraph_markers_list', 'sanity_1', 'split', 'structural_fts_as_text_combined', 'token_type_ids'],
        num_rows: 1088
    })
    test: Dataset({
        features: ['adu_cls_indices', 'attention_mask', 'cls_indices', 'component_cls_indices', 'essay_nr', 'feature_cls_indices', 'features_cls_indices', 'full_paragraph_w_cls', 'input_ids', 'label', 'paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_labels_numeric', 'paragraph_markers_list', 'sanity_1', 'split', 'structural_fts_as_text_combined', 'token_type_ids'],
        num_rows: 358
    })
    validation: Dataset({
        features: ['adu_cls_indices', 'attention_mask', 'cls_indic

In [22]:
dataset['test'].features['cls_indices'] = datasets.Array2D(shape=(12, 2), dtype="int32")
dataset['train'].features['cls_indices'] = datasets.Array2D(shape=(12, 2), dtype="int32")
dataset['validation'].features['cls_indices'] = datasets.Array2D(shape=(12, 2), dtype="int32")

In [23]:
dataset = dataset.map(lambda batch: batch, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'cls_indices', 'label'])

In [30]:
dataset['train']['full_paragraph_w_cls'][498]

'[CLS] Computers have revolutionized our lifestyle during last century. However, some people especially young ones use computers extremely and this excessive usage lead to increase stress among them, but computers have brought many advantages to our lives. From my point of view, computers have an important and inevitable role in economic, business, and other part of societies. [CLS] 1, Yes, No, Yes, No [CLS] In addition, they make the way of living easier by doing our routine duties. [CLS] 1, No, Yes, Yes, No'

In [28]:
dataset['train']['cls_indices'][498]

tensor([[ 0, 66],
        [76, 93],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1]])

In [29]:
dataset['train'][498]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0

In [32]:
for i,x in enumerate(tokenizer.tokenize(dataset['train']['full_paragraph_w_cls'][498])):
    print(i,x)

0 [CLS]
1 computers
2 have
3 revolution
4 ##ized
5 our
6 lifestyle
7 during
8 last
9 century
10 .
11 however
12 ,
13 some
14 people
15 especially
16 young
17 ones
18 use
19 computers
20 extremely
21 and
22 this
23 excessive
24 usage
25 lead
26 to
27 increase
28 stress
29 among
30 them
31 ,
32 but
33 computers
34 have
35 brought
36 many
37 advantages
38 to
39 our
40 lives
41 .
42 from
43 my
44 point
45 of
46 view
47 ,
48 computers
49 have
50 an
51 important
52 and
53 inevitable
54 role
55 in
56 economic
57 ,
58 business
59 ,
60 and
61 other
62 part
63 of
64 societies
65 .
66 [CLS]
67 1
68 ,
69 yes
70 ,
71 no
72 ,
73 yes
74 ,
75 no
76 [CLS]
77 in
78 addition
79 ,
80 they
81 make
82 the
83 way
84 of
85 living
86 easier
87 by
88 doing
89 our
90 routine
91 duties
92 .
93 [CLS]
94 1
95 ,
96 no
97 ,
98 yes
99 ,
100 yes
101 ,
102 no


In [26]:
### Work

### Get outputs at CLS indices for both ADU and Features

In [27]:
def get_cls_outputs(outputs, cls_indices):
    
    batch_size = cls_indices.shape[0]    
    nr_adus = cls_indices.shape[1]
    
    cls_adu_indices = cls_indices[:,:,0]
    cls_features_indices = cls_indices[:,:,1]
    
    outputs_adu_cls_indices = outputs[:,cls_adu_indices,:]
    outputs_features_cls_indices = outputs[:,cls_features_indices,:]
    
    outputs_adus = torch.cat([outputs_adu_cls_indices[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_adus = outputs_adus.reshape(batch_size, nr_adus, -1)
    
    outputs_features = torch.cat([outputs_features_cls_indices[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_features = outputs_features.reshape(batch_size, nr_adus, -1)
    
    return outputs_adus, outputs_features

## span representation function

In [28]:
# def minus_one(t):
    
#     return torch.where(t == 0, 0, t-1)

# def plus_one(t):
    
#     return torch.where(t >= MAX_SPAN, MAX_SPAN, t+1)

## custom BERT model

In [29]:
class CustomBERTKuri(nn.Module):

    def __init__(self, first_model, model_adu, model_fts, nr_classes):
        
        super(CustomBERTKuri, self).__init__()
        
        self.first_model = first_model
        
#         self.intermediate_linear_am = nn.Linear(3072, 768)
#         self.intermediate_linear_ac = nn.Linear(3072, 768) 
#         self.intermediate_linear_fts = nn.Linear(3072, 768)
        
        self.model_adu = model_adu
        self.model_fts = model_fts
        
        self.nr_classes = nr_classes
                
        # self.fc = nn.Linear(self.model_am.config.hidden_size + self.model_ac.config.hidden_size + self.model_fts.config.hidden_size, self.nr_classes)
        self.fc = nn.Linear(self.model_adu.config.hidden_size + self.model_fts.config.hidden_size, self.nr_classes)

    def forward(self, inputs):
        
        batch_tokenized, batch_cls_indices = inputs 
        # outputs = self.first_model(batch_tokenized, output_hidden_states=True)[1][12]
        outputs = self.first_model(batch_tokenized)[0]
        adu_cls_outputs, fts_cls_outputs = get_cls_outputs(outputs, batch_cls_indices)


#         am_minus_representations = self.intermediate_linear_am(am_minus_representations)
#         ac_minus_representations = self.intermediate_linear_ac(ac_minus_representations)
#         fts_minus_representations = self.intermediate_linear_fts(fts_minus_representations)


        output_model_adu = self.model_adu(inputs_embeds = adu_cls_outputs)[0]
        output_model_fts = self.model_fts(inputs_embeds = fts_cls_outputs)[0]
        
        #output_model_fts = self.model_fts(inputs_embeds = fts_minus_representations)[0]

        adu_representations = torch.cat([output_model_adu, output_model_fts], dim=-1)
        output = self.fc(adu_representations)

        return output

## Run

In [30]:
NB_EPOCHS = 40
BATCH_SIZE = 32

In [31]:
# first_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
# first_model.load_state_dict(torch.load('/notebooks/KURI-BERT/notebooks/full_formula_w_fts/icann_finetuned_work/finetuned_model.pth'))

In [32]:
first_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [33]:
model_adu = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [34]:
model_fts = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [35]:
custom_model = CustomBERTKuri(first_model, model_adu, model_fts, 3)

In [36]:
custom_model.to(device)

CustomBERTKuri(
  (first_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [37]:
loss = nn.CrossEntropyLoss(ignore_index=- 100)

In [38]:
optimizer = torch.optim.AdamW(custom_model.parameters(), lr=0.10000000000000002)

In [39]:
# 1.8738174228603844e-05
# 0.008111308307896872
# best learning rate found by the whole leslie business
# new best LR found= 9.999999999999997e-06

In [40]:
NR_BATCHES = len(dataset['train']) / BATCH_SIZE
num_training_steps = NB_EPOCHS * NR_BATCHES
num_warmup_steps = int(0.2 * num_training_steps)

In [41]:
def lr_lambda(current_step: int):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

In [42]:
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

### create dataloaders

In [43]:
train_dataloader = DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset['validation'], batch_size=BATCH_SIZE, shuffle=True)

In [44]:
def flatten_list(list_of_lists):
    return [x for sublist in list_of_lists for x in sublist]

In [45]:
def remove_dummy_labels(test_preds, test_labels):
    
    idxes = []
    test_labels_l = []
    for idx, val in enumerate(test_labels):
        if val != -100:
            idxes.append(idx)
            test_labels_l.append(val)
    
    test_preds_l = []
    for idx, val in enumerate(test_preds):
        for good_idx in idxes:
            if idx == good_idx:
                test_preds_l.append(val)
        
    return test_preds_l, test_labels_l

### training loop

In [46]:
from datasets import load_metric

In [47]:
import random
from torch_lr_finder import LRFinder

### LR Finder Leslie Smith 

## training 

In [48]:
def train(model, loss=None, optimizer=None, train_dataloader=None, val_dataloader=None, nb_epochs=20):
    """Training loop"""

    min_f1 = -torch.inf
    train_losses = []
    val_losses = []

    # Iterrate over epochs
    for e in range(nb_epochs):

        # Training
        train_loss = 0.0

        for i, batch in enumerate(tqdm(train_dataloader)):            
            
            #print(i)
            # unpack batch             
            labels = batch['label'].to(device)
            cls_indices = batch['cls_indices'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, cls_indices
            
            # Reset gradients to 0
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(inputs)
            
            # Compute training loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            train_loss += current_loss.detach().item()

            # Compute gradients
            current_loss.backward()

            # Update weights
            optimizer.step()            
            
            del batch
        
        scheduler.step()
            
        
        # Validation
        val_loss = 0.0
        
        # torch.cuda.empty_cache()
        
        # Put model in eval mode
        model.eval()
        
        preds_l = []
        labels_l = []
        
        for batch in tqdm(val_dataloader):            
            
            # unpack batch             
            labels = batch['label'].to(device)
            cls_indices = batch['cls_indices'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, cls_indices
            
            # Forward Pass
            outputs = model(inputs)

            # Compute validation loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            val_loss += current_loss.detach().item()
            
            preds_for_f1 = torch.argmax(outputs, dim=2).flatten().tolist()
            labels_for_f1 = labels.flatten().tolist()
            
            preds_l.append(preds_for_f1)
            labels_l.append(labels_for_f1)
            
            del batch
        
        # Prints
        
        preds_l = flatten_list(preds_l)
        labels_l = flatten_list(labels_l)
        
        preds_l, labels_l = remove_dummy_labels(preds_l, labels_l)
        
        f1_score_epoch = f1_score(preds_l, labels_l, average='macro')        
        
        print(f"Epoch {e+1}/{nb_epochs} \
                \t Training Loss: {train_loss/len(train_dataloader):.3f} \
                \t Validation Loss: {val_loss/len(val_dataloader):.3f} \
                \t F1 score: {f1_score_epoch}")
        
        train_losses.append(train_loss/len(train_dataloader))
        val_losses.append(val_loss/len(val_dataloader))
        

        # Save model if val loss decreases
        if f1_score_epoch > min_f1:

            min_f1 = f1_score_epoch
            torch.save(model.first_model.state_dict(), 'first_model.pt')
            torch.save(model.model_adu.state_dict(), 'model_adu.pt')
            torch.save(model.model_fts.state_dict(), 'model_fts.pt')
            # torch.save(model.model_fts.state_dict(), 'model_fts.pt')
            torch.save(model.state_dict(), 'best_model.pt')
            
    return train_losses, val_losses

In [49]:
train_losses, val_losses = train(custom_model, loss, optimizer, train_dataloader, val_dataloader, NB_EPOCHS)

100%|██████████| 9/9 [00:02<00:00,  3.02it/s]


Epoch 1/40                 	 Training Loss: 1.110                 	 Validation Loss: 1.117                 	 F1 score: 0.21861750837768823


100%|██████████| 9/9 [00:03<00:00,  2.99it/s]


Epoch 2/40                 	 Training Loss: 2.476                 	 Validation Loss: 0.882                 	 F1 score: 0.2613299449385853


100%|██████████| 9/9 [00:03<00:00,  2.97it/s]


...

Epoch 38/40                 	 Training Loss: 0.909                 	 Validation Loss: 0.880                 	 F1 score: 0.2613299449385853


100%|██████████| 9/9 [00:03<00:00,  2.97it/s]


Epoch 39/40                 	 Training Loss: 0.905                 	 Validation Loss: 0.876                 	 F1 score: 0.2613299449385853


100%|██████████| 9/9 [00:03<00:00,  2.99it/s]


Epoch 40/40                 	 Training Loss: 0.915                 	 Validation Loss: 0.881                 	 F1 score: 0.2613299449385853


### Predictions

In [50]:
test_dataloader = DataLoader(dataset['test'], batch_size=BATCH_SIZE, shuffle=True)

In [51]:
# first_model =  BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
# first_model.load_state_dict(torch.load('first_model.pt'))

In [52]:
first_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
first_model.load_state_dict(torch.load('first_model.pt'))

<All keys matched successfully>

In [53]:
model_adu = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_adu.load_state_dict(torch.load('model_adu.pt'))

<All keys matched successfully>

In [54]:
model_fts = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_fts.load_state_dict(torch.load('model_fts.pt'))

<All keys matched successfully>

In [55]:
# Load best model

custom_model_2 = CustomBERTKuri(first_model, model_adu, model_fts, 3)
custom_model_2.load_state_dict(torch.load('best_model.pt'))

custom_model_2.to(device).eval()

CustomBERTKuri(
  (first_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [56]:
def predict(model, test_dataloader=None):
    
    """Prediction loop"""

    preds_l = []
    labels_l = []
    
    model.eval()

    for batch in test_dataloader:            
            
        # unpack batch             
        labels = batch['label'].to(device).flatten().tolist()
        cls_indices = batch['cls_indices'].to(device)
        input_ids = batch['input_ids'].to(device)
        
        inputs = input_ids, cls_indices

        # get output
        
        raw_preds = model(inputs).to('cpu')
        # print(raw_preds.shape)
        raw_preds = raw_preds.detach()#.numpy()

        # Compute argmax
        
        predictions = torch.argmax(raw_preds, dim=2).flatten().tolist()
        preds_l.append(predictions)
        labels_l.append(labels)        
        
        del batch
            
    return flatten_list(preds_l), flatten_list(labels_l)

In [57]:
test_preds, test_labels = predict(custom_model_2, test_dataloader)

In [58]:
print(classification_report(test_labels, test_preds, digits=3))

              precision    recall  f1-score   support

        -100      0.000     0.000     0.000      3033
           0      0.000     0.000     0.000       155
           1      0.000     0.000     0.000       303
           2      0.187     1.000     0.316       805

    accuracy                          0.187      4296
   macro avg      0.047     0.250     0.079      4296
weighted avg      0.035     0.187     0.059      4296



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
test_preds_l, test_labels_l = remove_dummy_labels(test_preds, test_labels)

In [60]:
print(classification_report(test_labels_l, test_preds_l, digits=3))

              precision    recall  f1-score   support

           0      0.000     0.000     0.000       155
           1      0.000     0.000     0.000       303
           2      0.637     1.000     0.779       805

    accuracy                          0.637      1263
   macro avg      0.212     0.333     0.260      1263
weighted avg      0.406     0.637     0.496      1263



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
